# Dataset Preparation

## Dataset Download.

In [ ]:
#!wget -r -N -c -np https://physionet.org/files/eegmmidb/1.0.0/
!pip uninstall tensorflow -y


In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras

## Project dependencies export

In [ ]:
# import sys

# !pip install -q --upgrade pip
# !pip install -q ipykernel

# # 搜集当前notebook中的依赖
# !pip install -q nbconvert
# !jupyter nbconvert --to script EEG-User-Identification.ipynb

# with open('EEG-User-Identification.py', 'r') as f:
#     lines = f.readlines()

# ! pip install pipreqs
# !python -m  pipreqs.pipreqs ./



## Raw data clipping

### 原始eegmmidb下载数据集分割处理

In [1]:
# import necessary packages*+--
import numpy as np
import mne
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Reshape

2023-12-12 01:28:04.311428: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 01:28:07.716159: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 01:28:16.135442: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### data loading

In [ ]:
# data loading ...
# %cd /physionet.org/files/eegmmidb/1.0.0/

SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]
Lists = []
for j in SList:
    name = 's'+str(j)
    name = np.zeros((64, 160))
    name = np.expand_dims(name, axis=0)
    for i in RList:
        file = os.getcwd() + '/S'+str(j)+'/S'+str(j)+'R'+str(i)+'.edf'
        data = mne.io.read_raw_edf(file)
        raw_data = data.get_data()

        # 1 sec parts
        for k in range(int((np.size(raw_data,1))/160)):
          name = np.concatenate((name, np.expand_dims(raw_data[:, (k)*160:(k+1)*160], axis=0)), axis=0)

        # free up memory
        path = []
        cwd = os.getcwd()
        file = os.getcwd()+'/S'+str(j)+'/S'+str(j)+'R'+str(i)+'.edf'
        path.append(os.path.join(cwd, file))
        os.remove(path[0])
    name = name[1:np.size(name, 0), :, :]
    subj = 'S'+str(j)
    np.save(subj, name)


#### Channel selection

In [ ]:
# Selecting Channels
SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]

# For 3 Channels
channels3 = [8,10,12]
for j in SList:
  name = 'S'+str(j)+'.npy'
  a = np.load(name)
  a = a[:, channels3, :]
  subject = 's'+str(j)
  np.save(subject, a)

In [ ]:
# Selecting Channels
SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]

# For 4 Channels
channels4 = [31,35,48,52]
for j in SList:
  name = 'S'+str(j)+'.npy'
  a = np.load(name)
  a = a[:, channels4, :]
  subject = 's'+str(j)
  np.save(subject, a)

In [ ]:
# Selecting Channels
SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]

# For 16 Channels
channels16 = [8,10,12,21,23,29,31,35,37,40,41,46,48,50,52,54]
for j in SList:
  name = 'S'+str(j)+'.npy'
  a = np.load(name)
  a = a[:, channels16, :]
  subject = 's'+str(j)
  np.save(subject, a)

In [ ]:
# Selecting Channels
SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]

# For 32 Channels
channels32 = [0,2,3,4,6,8,10,12,14,16,17,18,20,21,22,23,26,29,31,33,35,37,40,41,46,48,50,52,54,57,60,62]
for j in SList:
  name = 'S'+str(j)+'.npy'
  a = np.load(name)
  a = a[:, channels32, :]
  subject = 's'+str(j)
  np.save(subject, a)

In [ ]:
# Assigning p% of data for training
SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]
p = 0.9
for j in SList:
  name = 's'+str(j)+'.npy'
  a = np.load(name)
  shuffle = list(range(np.size(a, 0)))
  np.random.shuffle(shuffle)
  a = a[shuffle, :, :]
  num_tr = int(p*np.size(a, 0))
  x_tr = a[0:num_tr, :, :]
  x_te = a[num_tr+1:np.size(a,0), :, :]
  subject_tr = 's'+str(j)+'_train'
  subject_te = 's'+str(j)+'_test'
  np.save(subject_tr, x_tr)
  np.save(subject_te, x_te)
  print(j)

#### Train set and Test set

##### 非图像版本

In [ ]:
import numpy as np
from tqdm import tqdm
SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]


# note that n_ch should be equal to what you selected in channel selection step!
# here we chose 4
n_ch = 32
y_train = np.zeros((1, 1))
x_train = np.zeros((1, n_ch, 160))
print("Train Dataset----------------------------------")
for j in tqdm(SList):
    name = 's'+str(j)+'_train.npy'
    a = np.load(name)
    #print(a.shape)
    x_train = np.concatenate((x_train, a), axis=0)
    y = (int(j)-1)*np.ones((np.size(a, 0), 1))
    y_train = np.concatenate((y_train, y))
    # print(j)
    del a
    del y

print("Test Dataset----------------------------------")
# note that n_ch should be equal to what you selected in channel selection step!
# here we chose 4
y_test = np.zeros((1, 1))
x_test = np.zeros((1, n_ch, 160))
for j in tqdm(SList):
    name = 's'+str(j)+'_test.npy'
    a = np.load(name)
    x_test = np.concatenate((x_test, a), axis=0)
    y = (int(j)-1)*np.ones((np.size(a, 0), 1))
    y_test = np.concatenate((y_test, y))
    # print(j)
    del a
    del y


##### 图像版本(本想在这里直接处理掉数据集的图像化，但是需要接近100GB内存才能放得下。暂时放弃。)

In [ ]:
import numpy as np
from tqdm import tqdm

SList = ["%03d"%i for i in range(1,110)]
RList = ["%02d"%i for i in range(1,15)]

# note that n_ch should be equal to what you selected in channel selection step!
n_ch = 32
y_train = np.zeros((1, 1))
x_train = np.zeros((1, 224, 224,3))
for j in tqdm(SList):
    name = 's'+str(j)+'_train.npy'
    a = np.load(name)
    a = a[..., np.newaxis]
    # 转化为tensor
    a_tensor = tf.convert_to_tensor(a, dtype=tf.float32)
    # 将后两个维度按照比例缩放到 224 x 224
    a_resized = tf.image.resize(a_tensor, [224, 224])
    # 转回numpy数组
    a_resized = a_resized.numpy()
    a = np.repeat(a_resized, 3, axis=-1)
    # print(a.shape)
    x_train = np.concatenate((x_train, a), axis=0)
    y = (int(j)-1)*np.ones((np.size(a, 0), 1))
    y_train = np.concatenate((y_train, y))
    del a,a_tensor,a_resized,y

y_test = np.zeros((1, 1))
x_test = np.zeros((1, 224, 224,3))
for j in tqdm(SList):
    name = 's'+str(j)+'_test.npy'
    a = np.load(name)
    a = a[..., np.newaxis]
    # 转化为tensor
    a_tensor = tf.convert_to_tensor(a, dtype=tf.float32)
    # 将后两个维度按照比例缩放到 224 x 224
    a_resized = tf.image.resize(a_tensor, [224, 224])
    # 转回numpy数组
    a_resized = a_resized.numpy()
    a = np.repeat(a_resized, 3, axis=-1)
    # print(a.shape)
    x_test = np.concatenate((x_test, a), axis=0)
    y = (int(j)-1)*np.ones((np.size(a, 0), 1))
    y_test = np.concatenate((y_test, y))
    del a,a_tensor,a_resized,y


#### Data preparation

#### 图像格式处理转换

In [ ]:
import tensorflow as tf
import numpy as np
# 存储为npy文件
np.save('EEG-32c-x_train.npy', x_train)
np.save('EEG-32c-x_test.npy', x_test)
np.save('EEG-32c-y_train.npy', y_train)
np.save('EEG-32c-y_test.npy', y_test)

In [ ]:
# 供直接加载使用
x_train = np.load('EEG-32c-x_train.npy')
y_train = np.load('EEG-32c-y_train.npy')
x_test = np.load('EEG-32c-x_test.npy')
y_test = np.load('EEG-32c-y_test.npy')

# deleting one unwanted index
x_train = x_train[1:np.size(x_train, 0), :, :]
x_test = x_test[1:np.size(x_test, 0), :, :]
y_train = y_train[1:np.size(y_train, 0), :]
y_test = y_test[1:np.size(y_test, 0), :]

# data shape
print('Shape of x_train:', np.shape(x_train))
print('Shape of x_test:', np.shape(x_test))
print('Shape of y_train:', np.shape(y_train))
print('Shape of y_test:', np.shape(y_test))
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape:', input_shape)

# normalizing your data
maxMat = np.amax(x_train, axis=0, keepdims=True)
x_train = np.divide(x_train, maxMat)
x_test = np.divide(x_test, maxMat)
del maxMat

# shuffling data
shuffle = list(range(np.size(x_train, 0)))
np.random.shuffle(shuffle)
x_train = x_train[shuffle, :, :]
y_train = y_train[shuffle, ]

shuffle = list(range(np.size(x_test, 0)))
np.random.shuffle(shuffle)
x_test = x_test[shuffle, :, :]
y_test = y_test[shuffle, ]

print('Shape of x_train after shuffling:', np.shape(x_train))
print('Shape of x_test after shuffling:', np.shape(x_test))
print('Shape of y_train after shuffling:', np.shape(y_train))
print('Shape of y_test after shuffling:', np.shape(y_test))
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape after shuffling:', input_shape)

del shuffle

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

num_classes = 109
y_train = np.reshape(y_train, [np.size(y_train, 0), ])
y_train = to_categorical(y_train, num_classes)
y_test = np.reshape(y_test, [np.size(y_test, 0), ])
y_test = to_categorical(y_test, num_classes)

print('Shape of x_train after reshaping:', np.shape(x_train))
print('Shape of x_test after reshaping:', np.shape(x_test))
print('Shape of y_train after reshaping:', np.shape(y_train))
print('Shape of y_test after reshaping:', np.shape(y_test))
input_shape = (x_train.shape[1], x_train.shape[2], 1)
print('Input shape after reshaping:', input_shape)


In [ ]:
print(y_test[1])

这里的input shape之所以是(n_ch, 160, 1)而不是(n_ch, 160)，感觉是为了跟图像识别领域的三个通道对应。

##### EEG to PIC

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 从 x_train 中提取出一个 32x160 的 tensor生成一个32x160的灰度图像
tensor = x_train[0]

# 在最后一维添加一维，转换为 32x160x1 的 tensor
tensor = np.expand_dims(tensor, axis=-1)
gray_img = tensor

# 显示灰度图像
plt.imshow(gray_img.squeeze(), cmap='gray')
plt.show()


In [ ]:
for i in range(10):
    # 从 x_train 中提取出一个 32x160 的 tensor生成一个32x160的灰度图像
    tensor = x_train[i]

    # 在最后一维添加一维，转换为 32x160x1 的 tensor
    # tensor = np.expand_dims(tensor, axis=-1)

    a_tensor = tf.convert_to_tensor(tensor, dtype=tf.float32)
    # 将后两个维度按照比例缩放到 224 x 224
    a_resized = tf.image.resize(a_tensor, [224, 224])
    # 转回numpy数组
    a_resized = a_resized.numpy()

    gray_img = a_resized

    # 显示灰度图像
    plt.imshow(gray_img.squeeze() , cmap='gray')
    plt.show()


### M3CV database

####  静态加载

In [ ]:
import numpy as np
# 供直接加载使用
X_train = np.load('Baidu-X_train.npy')
y_train = np.load('Baidu-X_train.npy')
X_val = np.load('Baidu-X_val.npy')
y_val = np.load('Baidu-y_val.npy')

In [ ]:
# import necessary packages*+--
import numpy as np
import mne
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Reshape

2023-05-31 16:33:00.219414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import scipy.io as scio
import pandas as pd
from sklearn.utils import shuffle

os.chdir('/home/u200810216/jupyter/')

# 读取数据
train_images = pd.read_csv('data/data151025/Enrollment_Info.csv')
val_images = pd.read_csv('data/data151025/Calibration_Info.csv')

train_images = shuffle(train_images, random_state=0)
val_images = shuffle(val_images)
# 划分训练集和校验集

train_image_list = train_images
val_image_list = val_images

df = train_image_list
train_img_list = train_image_list['EpochID'].values
train_label_list = train_image_list['SubjectID'].values


val_img_list = val_image_list['EpochID'].values
val_label_list = val_image_list['SubjectID'].values

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy.io as scio
from tensorflow.data import Dataset
from tqdm import tqdm

class MyDataset:
    def __init__(self, train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=10):
        self.mode = mode
        self.train_images = train_img_list[:n_samples]
        self.test_images = val_img_list[:n_samples]
        self.train_label = train_label_list[:n_samples]
        self.test_label = val_label_list[:n_samples]

    def load_eeg(self, eeg_path):
        data = scio.loadmat(eeg_path)
        return data['epoch_data']

    def __len__(self):
        if self.mode == 'train':
            return len(self.train_images)
        else:
            return len(self.test_images)

    def get_data(self):
        if self.mode == 'train':
            image_list = self.train_images
            label_list = self.train_label
        else:
            image_list = self.test_images
            label_list = self.test_label

        eeg_data = []
        eeg_labels = []

        for img, la in tqdm(zip(image_list, label_list), total=len(image_list), desc="Loading EEG data"):
            if self.mode == 'train':
                eeg_path = os.path.join('data/data151025/train/', img + '.mat')
            else:
                eeg_path = os.path.join('data/data151025/val/', img + '.mat')

            data = self.load_eeg(eeg_path)
            eeg_data.append(data)
            eeg_labels.append(int(la[4:]) - 1)

        eeg_data = np.array(eeg_data).astype('float32')
        eeg_data = eeg_data[..., np.newaxis]
        eeg_labels = np.array(eeg_labels).astype('int32')

        return eeg_data, eeg_labels
n_samples =  None
# n_samples =  100
train_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=n_samples)
val_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='test', n_samples=n_samples)

X_train, y_train = train_dataset.get_data()
X_val, y_val = val_dataset.get_data()


# # Convert the data to TensorFlow Dataset
# train_data = Dataset.from_tensor_slices((X_train, y_train))
# val_data = Dataset.from_tensor_slices((X_val, y_val))

# # Shuffle and batch the dataa
# train_data = train_data.shuffle(buffer_size=len(train_dataset)).batch(32)
# val_data = val_data.batch(32)


In [ ]:
train_data

In [ ]:
import tensorflow as tf
import numpy as np
# 存储为npy文件
np.save('Baidu-X_train.npy', X_train)
np.save('Baidu-X_val.npy', X_val)
np.save('Baidu-y_train.npy', y_train)
np.save('Baidu-y_val.npy', y_val)

In [ ]:
!pwd

#### 动态加载

In [ ]:
import numpy as np
import mne
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Reshape
import os
import numpy as np
import scipy.io as scio
import pandas as pd
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
# os.chdir('/home/u200810216/jupyter/')
os.chdir('/home/u200810216/jupyter/kaggle_m3cv')
# # 读取数据
# train_images = pd.read_csv('data/data151025/Enrollment_Info.csv')
# val_images = pd.read_csv('data/data151025/Calibration_Info.csv')

# 读取数据
train_images = pd.read_csv('Enrollment_Info.csv')
val_images = pd.read_csv('Calibration_Info.csv')

# print(train_images.head(5))


train_images = shuffle(train_images, random_state=0)
val_images = shuffle(val_images)
# 划分训练集和校验集

train_image_list = train_images
val_image_list = val_images

df = train_image_list
train_img_list = train_image_list['EpochID'].values
# train_label_list = train_image_list['SubjectID'].values
print(train_image_list.head())
train_label_list = train_image_list['subject'].values


val_img_list = val_image_list['EpochID'].values
val_label_list = val_image_list['SubjectID'].values
# print(val_image_list.head())
# val_label_list = val_image_list['subject'].values
print(train_img_list,train_label_list)

2023-05-31 16:57:13.379445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 16:57:14.385289: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-31 16:57:14.385425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-31 16:57:14.385433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

           EpochID subject  session  condition  usage
44959  epoch044960  sub074        1          7      1
26978  epoch026979  sub045        1          3      1
35644  epoch035645  sub059        1         10      1
36966  epoch036967  sub061        1          8      1
50637  epoch050638  sub084        1         12      1
['epoch044960' 'epoch026979' 'epoch035645' ... 'epoch042614' 'epoch043568'
 'epoch002733'] ['sub074' 'sub045' 'sub059' ... 'sub070' 'sub072' 'sub005']


In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy.io as scio
from tensorflow.data import Dataset
from tqdm import tqdm

class MyDataset:
    def __init__(self, train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=10):
        self.mode = mode
        self.train_images = train_img_list[:n_samples]
        self.test_images = val_img_list[:n_samples]
        self.train_label = train_label_list[:n_samples]
        self.test_label = val_label_list[:n_samples]

    def load_eeg(self, eeg_path):
        data = scio.loadmat(eeg_path)
        return data['epoch_data']

    def __len__(self):
        if self.mode == 'train':
            return len(self.train_images)
        else:
            return len(self.test_images)
    def get_data_generator(self):
        if self.mode == 'train':
            image_list = self.train_images
            label_list = self.train_label
        else:
            image_list = self.test_images
            label_list = self.test_label

#         for img, la in tqdm(zip(image_list, label_list), total=len(image_list), desc="Loading EEG data"):
        for img, la in zip(image_list, label_list):
            if self.mode == 'train':
                eeg_path = os.path.join('/home/u200810216/jupyter/kaggle_m3cv/train/', img + '.mat')
            else:
                eeg_path = os.path.join('/home/u200810216/jupyter/kaggle_m3cv/val/', img + '.mat')

            data = self.load_eeg(eeg_path)
            data = data[..., np.newaxis]
#             label = (int(la[4:])-1)*np.ones((np.size(95, 0), 1))

            label = to_categorical((int(la[4:])-1), 95)
#             print(data,label)
            yield data, label

n_samples= None
# 创建MyDataset实例
train_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='train', n_samples=n_samples)
val_dataset = MyDataset(train_img_list, val_img_list, train_label_list, val_label_list, mode='test', n_samples=n_samples)




In [ ]:
train_data = Dataset.from_generator(
    train_dataset.get_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(65, 1000,1), dtype=tf.float32),
        tf.TensorSpec(shape=(95), dtype=tf.int32)
    )
)
val_data = Dataset.from_generator(
    val_dataset.get_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(65, 1000,1), dtype=tf.float32),
        tf.TensorSpec(shape=(95), dtype=tf.int32)
    )
)


In [ ]:
# train_data = train_data.shuffle(buffer_size=len(train_dataset)).batch(256)
train_data = train_data.batch(128)
val_data = val_data.batch(128)
val_data

<BatchDataset element_spec=(TensorSpec(shape=(None, 65, 1000, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 95), dtype=tf.int32, name=None))>

# Network Architecture

## Transfomer

In [ ]:
modelname = "Transformer"

# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, LSTM
# from keras.layers import Conv2D
# from keras.layers import BatchNormalization, Activation, Reshape

# model = Sequential()

# #layer1
# model.add(Conv2D(128, kernel_size=(n_ch, 1), padding="valid", input_shape=input_shape))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))


# ## Layer 2
# model.add(Conv2D(256, kernel_size=(128, 1), padding="valid"))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))


# ## Layer 3
# model.add(Conv2D(512, kernel_size=(256, 1), padding="valid"))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))

# model.add(Flatten())


# #output layer
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

# model.summary()

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

def scaled_dot_product_attention(query, key, value):
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    dk = tf.cast(tf.shape(key)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, value)
    return output, attention_weights

class MultiHeadAttention(layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = layers.Dense(d_model)
        self.wk = layers.Dense(d_model)
        self.wv = layers.Dense(d_model)

        self.dense = layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)
        return output

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        layers.Dense(dff, activation='relu'),
        layers.Dense(d_model)
    ])

class EncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training):
        attn_output = self.mha(x, x, x)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2
class Transformer(layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Transformer, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = layers.Dropout(rate)

    def call(self, x, training):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training)

        return x

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :] // 2,
                            d_model)

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates


# Model hyperparameters
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
input_vocab_size = 640
maximum_position_encoding = 640
dropout_rate = 0.1

# Build the model
inputs = tf.keras.Input(shape=(4, 160, 1))
x = layers.Flatten()(inputs)
x = Transformer(num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, dropout_rate)(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()



## Vision Transformer

**不同于简单的Audio classification或Image classification！而是介于两者之间**

### Preprocess

The next step is to load a ViT image processor to process the image into a tensor:

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"

To avoid overfitting and to make the model more robust, add some data augmentation to the training part of the dataset.
Here we use Keras preprocessing layers to define the transformations for the training data (includes data augmentation),
and transformations for the validation data (only center cropping, resizing and normalizing). You can use `tf.image`or
any other library you prefer.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

size = (224, 224)

As a final preprocessing step, create a batch of examples using `DefaultDataCollator`. Unlike other data collators in 🤗 Transformers, the
`DefaultDataCollator` does not apply additional preprocessing, such as padding.

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

### Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load an
evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load
the [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your `compute_metrics` function is ready to go now, and you'll return to it when you set up your training.

### Train

<Tip>

If you are unfamiliar with fine-tuning a model with Keras, check out the [basic tutorial](https://huggingface.co/docs/transformers/main/en/tasks/./training#train-a-tensorflow-model-with-keras) first!

</Tip>

To fine-tune a model in TensorFlow, follow these steps:
1. Define the training hyperparameters, and set up an optimizer and a learning rate schedule.
2. Instantiate a pre-treined model.
3. Convert a 🤗 Dataset to a `tf.data.Dataset`.
4. Compile your model.
5. Add callbacks and use the `fit()` method to run the training.
6. Upload your model to 🤗 Hub to share with the community.

Start by defining the hyperparameters, optimizer and learning rate schedule:

In [ ]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 5
num_train_steps = len(x_train) * num_epochs
learning_rate = 3e-5
weight_decay_rate = 0.01

optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0,
)

Then, load ViT with [TFAutoModelForImageClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModelForImageClassification) along with the label mappings:

In [ ]:
from transformers import TFAutoModelForImageClassification

model = TFAutoModelForImageClassification.from_pretrained(
    checkpoint,

)

Convert your datasets to the `tf.data.Dataset` format using the [to_tf_dataset](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.to_tf_dataset) and your `data_collator`:

In [ ]:
# 转换x_train和y_train为tf.data.Dataset格式
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

# 转换x_test和y_test为tf.data.Dataset格式
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# 对数据进行shuffle、batch等操作
tf_train_dataset = train_dataset.batch(batch_size)
tf_eval_dataset = test_dataset.batch(batch_size)

Configure the model for training with `compile()`:

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)

To compute the accuracy from the predictions and push your model to the 🤗 Hub, use [Keras callbacks](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/keras_callbacks).
Pass your `compute_metrics` function to [KerasMetricCallback](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/keras_callbacks#transformers.KerasMetricCallback),
and use the [PushToHubCallback](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/keras_callbacks#transformers.PushToHubCallback) to upload the model:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_eval_dataset)
# push_to_hub_callback = PushToHubCallback(
#     output_dir="EEG_classifier",
#     tokenizer=image_processor,
#     save_strategy="no",
# )
callbacks = [metric_callback]

Finally, you are ready to train your model! Call `fit()` with your training and validation datasets, the number of epochs,
and your callbacks to fine-tune the model:

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=num_epochs, callbacks=callbacks)

## Vision Transfomer 新实现(eegmmidb)

In [ ]:
import tensorflow as tf
from transformers import TFViTModel, ViTConfig
from tensorflow.keras.callbacks import ModelCheckpoint
os.chdir('/home/u200810216/jupyter/')

# 加载预先训练好的 ViT 模型的配置
config = ViTConfig.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=109)

# 修改配置，使其适用于单通道图像分类任务
config.input_channels = 1
config.hidden_dropout_prob = 0.0
config.attention_probs_dropout_prob = 0.0

# 加载预先训练好的 ViT 模型的权重
vit = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k", config=config)
#vit.save_pretrained("./vitmodel")

# 使用单通道图像进行分类
inputs = tf.keras.layers.Input(shape=(32, 160, 1))
x = tf.image.resize(inputs, (224, 224))
x = tf.keras.layers.Concatenate(axis=-1)([x, x, x])
x = tf.keras.layers.Lambda(lambda x: tf.transpose(x, [0, 3, 1, 2]))(x)
outputs = vit(x).last_hidden_state[:, 0, :]

# 添加输出层
outputs = tf.keras.layers.Dense(109, activation='softmax')(outputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)


In [ ]:
from transformers import create_optimizer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import load_model
batch_size = 16
num_epochs = 5
num_train_steps = len(x_train) * num_epochs
learning_rate = 3e-5
weight_decay_rate = 0.01
epochs = 100

optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0,
)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)


# 记录日志

log_dir = "logs/transfomer"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy',
                       'FalsePositives',
                       'FalseNegatives'])


# 定义一个回调函数来保存模型和历史记录
checkpoint_callback = ModelCheckpoint(
    filepath='transfomer.h5',
    save_weights_only=False,
    save_best_only=True,
    monitor='val_loss',
    verbose=1,
)


# 训练模型
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_split=0.2, callbacks=[tensorboard_callback, checkpoint_callback])

# 保存模型
model.save('logs/transfomer/vision_transfomer_test.h5')  # 保存为 .h5 文件格式


In [ ]:
# 保存模型
model.save('vision_transfomer_last.h5')  # 保存为 .h5 文件格式

## Vision Transfomer 新实现(百度数据集)

In [ ]:
import os
import tensorflow as tf
from transformers import TFViTModel, ViTConfig
from tensorflow.keras.callbacks import ModelCheckpoint
# os.chdir('/home/u200810216/jupyter/')
# tf.debugging.set_log_device_placement(True)

# 加载预先训练好的 ViT 模型的配置
config = ViTConfig.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=95)

# 修改配置，使其适用于单通道图像分类任务
config.input_channels = 1
config.hidden_dropout_prob = 0.0
config.attention_probs_dropout_prob = 0.0

In [ ]:
from transformers import create_optimizer
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

# 之后是您的其他代码


In [ ]:
# 使用tf.distribute.MirroredStrategy
strategy = tf.distribute.MirroredStrategy()
#

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
batch_size = 16
epochs = 5
num_train_steps = len(train_img_list) * epochs
learning_rate = 3e-5
weight_decay_rate = 0.01




# with strategy.scope():
    # 加载预先训练好的 ViT 模型的权重
vit = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k", config=config)
#vit.save_pretrained("./vitmodel")

# 使用单通道图像进行分类
inputs = tf.keras.layers.Input(shape=(65, 1000, 1))# 这里等待修改
x = tf.image.resize(inputs, (224, 224))
x = tf.keras.layers.Concatenate(axis=-1)([x, x, x])
x = tf.keras.layers.Lambda(lambda x: tf.transpose(x, [0, 3, 1, 2]))(x)
outputs = vit(x).last_hidden_state[:, 0, :]

# 添加输出层
outputs = tf.keras.layers.Dense(95, activation='softmax')(outputs)
# outputs = tf.keras.layers.Lambda(lambda x: tf.argmax(x, axis=1))(outputs)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=1,
)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
# loss = tf.keras.losses.SparseCategoricalCrossentropy()


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=['accuracy',
                       'FalsePositives',
                       'FalseNegatives'])


# 记录日志
log_dir = "logs/transfomer"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# 定义一个回调函数来保存模型和历史记录
checkpoint_callback = ModelCheckpoint(
    filepath='transfomer.h5',
    save_weights_only=True,
    save_best_only=False,
    monitor='val_loss',
    verbose=1,
)


All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
len(train_img_list)

In [ ]:
# 训练模型
history = model.fit(train_data,
                    epochs=25,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])
# 保存模型权重而不是整个模型
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights.h5')

In [ ]:
#
model.load_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights_val_1.h5')
history = model.fit(val_data,
                    epochs=2,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])

# 保存模型
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights_val_3.h5')  # 保存为 .h5 文件格式


Epoch 1/2
     96/Unknown - 108s 1s/step - loss: 0.3789 - accuracy: 0.8876 - false_positives: 794.0000 - false_negatives: 1815.0000
Epoch 1: saving model to transfomer.h5
96/96 [==============================] - 230s 2s/step - loss: 0.3789 - accuracy: 0.8876 - false_positives: 794.0000 - false_negatives: 1815.0000 - val_loss: 0.2092 - val_accuracy: 0.9446 - val_false_positives: 398.0000 - val_false_negatives: 949.0000
Epoch 2/2
96/96 [==============================] - ETA: 0s - loss: 0.1722 - accuracy: 0.9568 - false_positives: 311.0000 - false_negatives: 750.0000
Epoch 2: saving model to transfomer.h5
96/96 [==============================] - 233s 2s/step - loss: 0.1722 - accuracy: 0.9568 - false_positives: 311.0000 - false_negatives: 750.0000 - val_loss: 0.1037 - val_accuracy: 0.9767 - val_false_positives: 171.0000 - val_false_negatives: 421.0000


In [ ]:
model.load_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights_val_3.h5')
history = model.fit(train_data,
                    epochs=3,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])

# 保存模型
model.save_weights('/home/u200810216/jupyter/kaggle_m3cv/vision_transfomer_test_kaggle_batch128_weights_val_3_train_3.h5')  # 保存为 .h5 文件格式


Epoch 1/3
    452/Unknown - 1503s 3s/step - loss: 0.0604 - accuracy: 0.9852 - false_positives: 614.0000 - false_negatives: 1015.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 1755s 4s/step - loss: 0.0604 - accuracy: 0.9852 - false_positives: 614.0000 - false_negatives: 1015.0000 - val_loss: 2.9653 - val_accuracy: 0.4057 - val_false_positives: 5381.0000 - val_false_negatives: 7663.0000
Epoch 2/3
452/452 [==============================] - ETA: 0s - loss: 0.0021 - accuracy: 1.0000 - false_positives: 2.0000 - false_negatives: 2.0000
Epoch 2: saving model to transfomer.h5
452/452 [==============================] - 1017s 2s/step - loss: 0.0021 - accuracy: 1.0000 - false_positives: 2.0000 - false_negatives: 2.0000 - val_loss: 3.0420 - val_accuracy: 0.4031 - val_false_positives: 5474.0000 - val_false_negatives: 7679.0000
Epoch 3/3
452/452 [==============================] - ETA: 0s - loss: 0.0011 - accuracy: 1.0000 - false_positives: 0.0000e+00 - false_ne

In [ ]:
import os
epochs = 64

weights_path = '/home/u200810216/jupyter/kaggle_m3cv/'
initial_weights_file = 'vision_transfomer_test_kaggle_batch128_weights_val_3_train_3_roll_6.h5'
final_weights_file = f'vision_transfomer_test_kaggle_batch128_weights_val_3_train_3_roll_6_roll{epochs}.h5'

# 加载预训练权重
model.load_weights(os.path.join(weights_path, initial_weights_file))

# 训练循环
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    if epoch % 2 == 0:
        print("Training on train_data")
        data = train_data
    else:
        print("Training on val_data")
        data = val_data

    history = model.fit(data,
                        epochs=1,
                        verbose=1,
                        callbacks=[tensorboard_callback, checkpoint_callback])

    # 保存模型参数
    model.save_weights(os.path.join(weights_path, "{}_epoch_{}.h5".format(final_weights_file.replace(".h5",""), epoch+1)))

# 保存最终模型参数
model.save_weights(os.path.join(weights_path, final_weights_file))


Epoch 1/64
Training on train_data
    452/Unknown - 1362s 3s/step - loss: 0.0280 - accuracy: 0.9936 - false_positives: 261.0000 - false_negatives: 500.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 1369s 3s/step - loss: 0.0280 - accuracy: 0.9936 - false_positives: 261.0000 - false_negatives: 500.0000
Epoch 2/64
Training on val_data
     96/Unknown - 185s 2s/step - loss: 0.1188 - accuracy: 0.9651 - false_positives: 297.0000 - false_negatives: 555.0000
Epoch 1: saving model to transfomer.h5
96/96 [==============================] - 191s 2s/step - loss: 0.1188 - accuracy: 0.9651 - false_positives: 297.0000 - false_negatives: 555.0000
Epoch 3/64
Training on train_data
    452/Unknown - 716s 2s/step - loss: 0.0218 - accuracy: 0.9956 - false_positives: 191.0000 - false_negatives: 344.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 722s 2s/step - loss: 0.0218 - accuracy: 0.9956 - false_positives: 191.0000 - false_ne

     96/Unknown - 126s 1s/step - loss: 0.0090 - accuracy: 0.9973 - false_positives: 25.0000 - false_negatives: 35.0000
Epoch 1: saving model to transfomer.h5
96/96 [==============================] - 132s 1s/step - loss: 0.0090 - accuracy: 0.9973 - false_positives: 25.0000 - false_negatives: 35.0000
Epoch 51/64
Training on train_data
    452/Unknown - 626s 1s/step - loss: 0.0075 - accuracy: 0.9980 - false_positives: 100.0000 - false_negatives: 131.0000
Epoch 1: saving model to transfomer.h5
452/452 [==============================] - 631s 1s/step - loss: 0.0075 - accuracy: 0.9980 - false_positives: 100.0000 - false_negatives: 131.0000
Epoch 52/64
Training on val_data
     96/Unknown - 190s 2s/step - loss: 0.0093 - accuracy: 0.9975 - false_positives: 27.0000 - false_negatives: 35.0000
Epoch 1: saving model to transfomer.h5
96/96 [==============================] - 195s 2s/step - loss: 0.0093 - accuracy: 0.9975 - false_positives: 27.0000 - false_negatives: 35.0000
Epoch 53/64
Training on tr

In [ ]:
!pip install tensorflow-addons


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 858.0 kB/s eta 0:00:0000:0100:01


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
class CustomAdamW(tfa.optimizers.AdamW):
    @classmethod
    def from_config(cls, config, custom_objects=None):
        if "power" in config:
            config.pop("power")
        return super().from_config(config, custom_objects)

class WarmUp(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_schedule_fn, warmup_steps, **kwargs):
        super().__init__(**kwargs)
        self.initial_learning_rate = initial_learning_rate
        self.decay_schedule_fn = decay_schedule_fn
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step_float = tf.cast(step, tf.float32)
        warmup_steps_float = tf.cast(self.warmup_steps, tf.float32)
        warmup_percent_done = global_step_float / warmup_steps_float
        warmup_learning_rate = self.initial_learning_rate * warmup_percent_done

        return tf.cond(
            global_step_float < warmup_steps_float,
            lambda: warmup_learning_rate,
            lambda: self.decay_schedule_fn(step - self.warmup_steps),
        )

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_schedule_fn": self.decay_schedule_fn,
            "warmup_steps": self.warmup_steps,
        }

# 加载保存的模型
model = tf.keras.models.load_model('vision_transfomer_last.h5', custom_objects={'TFViTModel': TFViTModel, 'AdamWeightDecay': CustomAdamW, 'WarmUp': WarmUp})


TypeError: object.__init__() takes exactly one argument (the instance to initialize)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# 加载保存的模型
loaded_model = tf.keras.models.load_model('vision_transfomer_last.h5', custom_objects={'TFViTModel': TFViTModel, 'AdamWeightDecay': AdamWeightDecay})

# # (可选) 如果您想更改学习速率或优化器，您可以在这里设置
# optimizer = Adam(learning_rate=0.0001)
# loaded_model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                      metrics=['accuracy'])

# 继续训练
new_epochs = 20  # 设置新的训练轮数
history = loaded_model.fit(train_data,
                    epochs=new_epochs,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[tensorboard_callback, checkpoint_callback])

# 保存模型
model.save('logs/transfomer/vision_transfomer_test.h5')  # 保存为 .h5 文件格式



# 保存模型
model.save('vision_transfomer_last.h5')  # 保存为 .h5 文件格式
# 保存继续训练后的模型
loaded_model.save('vision_transfomer_trained_further.h5')  # 保存为 .h5 文件格式


NameError: name 'AdamWeightDecay' is not defined

In [ ]:
print(type(optimizer).__name__)


AdamWeightDecay


# Model Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

batch_size = 256
epochs = 10000

# 记录日志
log_dir = "logs/fit"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# 定义优化器
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.constant(d_model, dtype=tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):  # 添加 get_config 方法
        return {
            'd_model': self.d_model.numpy(),
            'warmup_steps': self.warmup_steps
        }


learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy',
                       'FalsePositives',
                       'FalseNegatives'])


# 定义一个回调函数来保存模型和历史记录
checkpoint_callback = ModelCheckpoint(
    filepath='transfomer.h5',
    save_weights_only=False,
    save_best_only=True,
    monitor='val_loss',
    verbose=1,
)


history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_split=0.2, callbacks=[tensorboard_callback, checkpoint_callback])

# 保存模型
model.save('transfomer_test.h5')  # 保存为 .h5 文件格式

# Model Evaluation

In [ ]:
os.chdir('/home/u200810216/jupyter/')

score_train = model.evaluate(x_train, y_train, verbose=0)
score_test = model.evaluate(x_test, y_test, verbose=0)
print("mean train accuracy is:   ", (score_train[1]))
print("mean test accuracy is:   ", (score_test[1]))

print("FAR is:   ", (score_train[2]+score_test[2])/(np.size(y_train,0)+np.size(y_test,0)))
print("FRR is:   ", (score_train[3]+score_test[3])/(np.size(y_train,0)+np.size(y_test,0)))

In [ ]:
! tensorboard --logdir logs/fit --port 6007

## Loss and Accuracy curves

In [ ]:
import matplotlib.pyplot as plt
# plt.style.use(['science','high-vis','grid'])
# "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid(True)
plt.savefig('vit_accuracy.png', dpi=600)
plt.show()

# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.grid(True)
plt.savefig('vit_accuracy.png.png', dpi=600)
plt.show()

In [ ]:
import re

# 输入的字符串
log_string = """
Epoch 1/100
WARNING:tensorflow:Gradients do not exist for variables ['tf_vi_t_model_2/vit/pooler/dense/kernel:0', 'tf_vi_t_model_2/vit/pooler/dense/bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss` argument?
WARNING:tensorflow:Gradients do not exist for variables ['tf_vi_t_model_2/vit/pooler/dense/kernel:0', 'tf_vi_t_model_2/vit/pooler/dense/bias:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss` argument?
7820/7821 [============================>.] - ETA: 0s - loss: 0.5048 - accuracy: 0.9068 - false_positives: 1999.0000 - false_negatives: 23218.0000
Epoch 1: val_loss improved from inf to 0.07635, saving model to transfomer.h5
7821/7821 [==============================] - 714s 90ms/step - loss: 0.5048 - accuracy: 0.9068 - false_positives: 1999.0000 - false_negatives: 23218.0000 - val_loss: 0.0764 - val_accuracy: 0.9802 - val_false_positives: 456.0000 - val_false_negatives: 797.0000
Epoch 2/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0489 - accuracy: 0.9874 - false_positives: 1222.0000 - false_negatives: 1896.0000
Epoch 2: val_loss improved from 0.07635 to 0.04349, saving model to transfomer.h5
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0489 - accuracy: 0.9874 - false_positives: 1222.0000 - false_negatives: 1896.0000 - val_loss: 0.0435 - val_accuracy: 0.9879 - val_false_positives: 318.0000 - val_false_negatives: 437.0000
Epoch 3/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0279 - accuracy: 0.9924 - false_positives: 800.0000 - false_negatives: 1116.0000
Epoch 3: val_loss improved from 0.04349 to 0.02177, saving model to transfomer.h5
7821/7821 [==============================] - 700s 89ms/step - loss: 0.0279 - accuracy: 0.9924 - false_positives: 800.0000 - false_negatives: 1116.0000 - val_loss: 0.0218 - val_accuracy: 0.9940 - val_false_positives: 148.0000 - val_false_negatives: 221.0000
Epoch 4/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0209 - accuracy: 0.9945 - false_positives: 583.0000 - false_negatives: 793.0000
Epoch 4: val_loss did not improve from 0.02177
7821/7821 [==============================] - 698s 89ms/step - loss: 0.0209 - accuracy: 0.9945 - false_positives: 583.0000 - false_negatives: 793.0000 - val_loss: 0.0221 - val_accuracy: 0.9947 - val_false_positives: 145.0000 - val_false_negatives: 180.0000
Epoch 5/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0167 - accuracy: 0.9956 - false_positives: 487.0000 - false_negatives: 613.0000
Epoch 5: val_loss did not improve from 0.02177
7821/7821 [==============================] - 699s 89ms/step - loss: 0.0167 - accuracy: 0.9956 - false_positives: 487.0000 - false_negatives: 613.0000 - val_loss: 0.0508 - val_accuracy: 0.9876 - val_false_positives: 354.0000 - val_false_negatives: 410.0000
Epoch 6/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0133 - accuracy: 0.9966 - false_positives: 388.0000 - false_negatives: 487.0000
Epoch 6: val_loss improved from 0.02177 to 0.01470, saving model to transfomer.h5
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0133 - accuracy: 0.9966 - false_positives: 388.0000 - false_negatives: 487.0000 - val_loss: 0.0147 - val_accuracy: 0.9957 - val_false_positives: 115.0000 - val_false_negatives: 148.0000
Epoch 7/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0106 - accuracy: 0.9972 - false_positives: 304.0000 - false_negatives: 372.0000
Epoch 7: val_loss did not improve from 0.01470
7821/7821 [==============================] - 698s 89ms/step - loss: 0.0106 - accuracy: 0.9972 - false_positives: 304.0000 - false_negatives: 372.0000 - val_loss: 0.0195 - val_accuracy: 0.9942 - val_false_positives: 158.0000 - val_false_negatives: 195.0000
Epoch 8/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0107 - accuracy: 0.9972 - false_positives: 317.0000 - false_negatives: 389.0000
Epoch 8: val_loss did not improve from 0.01470
7821/7821 [==============================] - 699s 89ms/step - loss: 0.0107 - accuracy: 0.9972 - false_positives: 317.0000 - false_negatives: 389.0000 - val_loss: 0.0196 - val_accuracy: 0.9954 - val_false_positives: 114.0000 - val_false_negatives: 160.0000
Epoch 9/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0084 - accuracy: 0.9979 - false_positives: 247.0000 - false_negatives: 280.0000
Epoch 9: val_loss improved from 0.01470 to 0.00727, saving model to transfomer.h5
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0084 - accuracy: 0.9979 - false_positives: 247.0000 - false_negatives: 280.0000 - val_loss: 0.0073 - val_accuracy: 0.9981 - val_false_positives: 51.0000 - val_false_negatives: 62.0000
Epoch 10/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0083 - accuracy: 0.9978 - false_positives: 246.0000 - false_negatives: 299.0000
Epoch 10: val_loss did not improve from 0.00727
7821/7821 [==============================] - 699s 89ms/step - loss: 0.0083 - accuracy: 0.9978 - false_positives: 246.0000 - false_negatives: 299.0000 - val_loss: 0.0103 - val_accuracy: 0.9969 - val_false_positives: 86.0000 - val_false_negatives: 101.0000
Epoch 11/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0078 - accuracy: 0.9981 - false_positives: 216.0000 - false_negatives: 256.0000
Epoch 11: val_loss improved from 0.00727 to 0.00611, saving model to transfomer.h5
7821/7821 [==============================] - 700s 89ms/step - loss: 0.0078 - accuracy: 0.9981 - false_positives: 216.0000 - false_negatives: 256.0000 - val_loss: 0.0061 - val_accuracy: 0.9985 - val_false_positives: 42.0000 - val_false_negatives: 49.0000
Epoch 12/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0073 - accuracy: 0.9982 - false_positives: 213.0000 - false_negatives: 240.0000
Epoch 12: val_loss did not improve from 0.00611
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0073 - accuracy: 0.9982 - false_positives: 213.0000 - false_negatives: 240.0000 - val_loss: 0.0113 - val_accuracy: 0.9967 - val_false_positives: 84.0000 - val_false_negatives: 115.0000
Epoch 13/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0054 - accuracy: 0.9986 - false_positives: 162.0000 - false_negatives: 187.0000
Epoch 13: val_loss did not improve from 0.00611
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0054 - accuracy: 0.9986 - false_positives: 162.0000 - false_negatives: 187.0000 - val_loss: 0.0072 - val_accuracy: 0.9979 - val_false_positives: 62.0000 - val_false_negatives: 68.0000
Epoch 14/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0060 - accuracy: 0.9983 - false_positives: 192.0000 - false_negatives: 221.0000
Epoch 14: val_loss did not improve from 0.00611
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0060 - accuracy: 0.9983 - false_positives: 192.0000 - false_negatives: 221.0000 - val_loss: 0.0084 - val_accuracy: 0.9980 - val_false_positives: 56.0000 - val_false_negatives: 68.0000
Epoch 15/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9988 - false_positives: 138.0000 - false_negatives: 159.0000
Epoch 15: val_loss did not improve from 0.00611
7821/7821 [==============================] - 700s 89ms/step - loss: 0.0051 - accuracy: 0.9988 - false_positives: 138.0000 - false_negatives: 159.0000 - val_loss: 0.0493 - val_accuracy: 0.9864 - val_false_positives: 365.0000 - val_false_negatives: 461.0000
Epoch 16/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0053 - accuracy: 0.9987 - false_positives: 156.0000 - false_negatives: 177.0000
Epoch 16: val_loss did not improve from 0.00611
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0053 - accuracy: 0.9987 - false_positives: 156.0000 - false_negatives: 177.0000 - val_loss: 0.0129 - val_accuracy: 0.9971 - val_false_positives: 82.0000 - val_false_negatives: 98.0000
Epoch 17/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0046 - accuracy: 0.9987 - false_positives: 143.0000 - false_negatives: 172.0000
Epoch 17: val_loss did not improve from 0.00611
7821/7821 [==============================] - 700s 89ms/step - loss: 0.0046 - accuracy: 0.9987 - false_positives: 143.0000 - false_negatives: 172.0000 - val_loss: 0.0067 - val_accuracy: 0.9984 - val_false_positives: 46.0000 - val_false_negatives: 57.0000
Epoch 18/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0046 - accuracy: 0.9990 - false_positives: 122.0000 - false_negatives: 141.0000
Epoch 18: val_loss did not improve from 0.00611
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0046 - accuracy: 0.9990 - false_positives: 122.0000 - false_negatives: 141.0000 - val_loss: 0.0076 - val_accuracy: 0.9982 - val_false_positives: 49.0000 - val_false_negatives: 63.0000
Epoch 19/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0041 - accuracy: 0.9989 - false_positives: 122.0000 - false_negatives: 148.0000
Epoch 19: val_loss did not improve from 0.00611
7821/7821 [==============================] - 700s 90ms/step - loss: 0.0041 - accuracy: 0.9989 - false_positives: 122.0000 - false_negatives: 148.0000 - val_loss: 0.0090 - val_accuracy: 0.9976 - val_false_positives: 69.0000 - val_false_negatives: 77.0000
Epoch 20/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0050 - accuracy: 0.9988 - false_positives: 138.0000 - false_negatives: 159.0000
Epoch 20: val_loss improved from 0.00611 to 0.00391, saving model to transfomer.h5
7821/7821 [==============================] - 702s 90ms/step - loss: 0.0050 - accuracy: 0.9988 - false_positives: 138.0000 - false_negatives: 159.0000 - val_loss: 0.0039 - val_accuracy: 0.9988 - val_false_positives: 30.0000 - val_false_negatives: 42.0000
Epoch 21/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9992 - false_positives: 99.0000 - false_negatives: 107.0000
Epoch 21: val_loss did not improve from 0.00391
7821/7821 [==============================] - 700s 89ms/step - loss: 0.0036 - accuracy: 0.9992 - false_positives: 99.0000 - false_negatives: 107.0000 - val_loss: 0.0395 - val_accuracy: 0.9936 - val_false_positives: 192.0000 - val_false_negatives: 212.0000
Epoch 22/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0036 - accuracy: 0.9991 - false_positives: 104.0000 - false_negatives: 112.0000
Epoch 22: val_loss did not improve from 0.00391
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0036 - accuracy: 0.9991 - false_positives: 104.0000 - false_negatives: 112.0000 - val_loss: 0.0060 - val_accuracy: 0.9986 - val_false_positives: 38.0000 - val_false_negatives: 49.0000
Epoch 23/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0026 - accuracy: 0.9992 - false_positives: 91.0000 - false_negatives: 102.0000
Epoch 23: val_loss did not improve from 0.00391
7821/7821 [==============================] - 703s 90ms/step - loss: 0.0026 - accuracy: 0.9992 - false_positives: 91.0000 - false_negatives: 102.0000 - val_loss: 0.0055 - val_accuracy: 0.9986 - val_false_positives: 39.0000 - val_false_negatives: 46.0000
Epoch 24/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0033 - accuracy: 0.9992 - false_positives: 99.0000 - false_negatives: 107.0000
Epoch 24: val_loss did not improve from 0.00391
7821/7821 [==============================] - 703s 90ms/step - loss: 0.0033 - accuracy: 0.9992 - false_positives: 99.0000 - false_negatives: 107.0000 - val_loss: 0.0108 - val_accuracy: 0.9973 - val_false_positives: 73.0000 - val_false_negatives: 86.0000
Epoch 25/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0034 - accuracy: 0.9992 - false_positives: 99.0000 - false_negatives: 113.0000
Epoch 25: val_loss did not improve from 0.00391
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0034 - accuracy: 0.9992 - false_positives: 99.0000 - false_negatives: 113.0000 - val_loss: 0.0054 - val_accuracy: 0.9988 - val_false_positives: 30.0000 - val_false_negatives: 40.0000
Epoch 26/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0027 - accuracy: 0.9993 - false_positives: 86.0000 - false_negatives: 95.0000
Epoch 26: val_loss did not improve from 0.00391
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0027 - accuracy: 0.9993 - false_positives: 86.0000 - false_negatives: 95.0000 - val_loss: 0.0095 - val_accuracy: 0.9976 - val_false_positives: 63.0000 - val_false_negatives: 78.0000
Epoch 27/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0028 - accuracy: 0.9992 - false_positives: 86.0000 - false_negatives: 94.0000
Epoch 27: val_loss did not improve from 0.00391
7821/7821 [==============================] - 700s 89ms/step - loss: 0.0028 - accuracy: 0.9992 - false_positives: 86.0000 - false_negatives: 94.0000 - val_loss: 0.0056 - val_accuracy: 0.9987 - val_false_positives: 38.0000 - val_false_negatives: 43.0000
Epoch 28/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0030 - accuracy: 0.9993 - false_positives: 89.0000 - false_negatives: 98.0000
Epoch 28: val_loss did not improve from 0.00391
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0030 - accuracy: 0.9993 - false_positives: 89.0000 - false_negatives: 98.0000 - val_loss: 0.0090 - val_accuracy: 0.9979 - val_false_positives: 65.0000 - val_false_negatives: 74.0000
Epoch 29/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0025 - accuracy: 0.9994 - false_positives: 70.0000 - false_negatives: 80.0000
Epoch 29: val_loss did not improve from 0.00391
7821/7821 [==============================] - 702s 90ms/step - loss: 0.0025 - accuracy: 0.9994 - false_positives: 70.0000 - false_negatives: 80.0000 - val_loss: 0.0055 - val_accuracy: 0.9985 - val_false_positives: 43.0000 - val_false_negatives: 49.0000
Epoch 30/100
7820/7821 [============================>.] - ETA: 0s - loss: 0.0023 - accuracy: 0.9994 - false_positives: 66.0000 - false_negatives: 75.0000
Epoch 30: val_loss did not improve from 0.00391
7821/7821 [==============================] - 701s 90ms/step - loss: 0.0023 - accuracy: 0.9994 - false_positives: 66.0000 - false_negatives: 75.0000 - val_loss: 0.0065 - val_accuracy: 0.9985 - val_false_positives: 40.0000 - val_false_negatives: 57.0000
"""

train_loss = []
train_accuracy = []
val_loss = []
val_accuracy = []

# 逐行处理字符串，提取出需要的数据
for line in log_string.split("\n"):
    if not line.strip():
        continue
    train_loss_match = re.search(r'loss: \[(.*)\]', line)
    if train_loss_match:
        train_loss.extend([float(x) for x in train_loss_match.group(1).split(',')])
    train_accuracy_match = re.search(r'accuracy: \[(.*)\]', line)
    if train_accuracy_match:
        train_accuracy.extend([float(x) for x in train_accuracy_match.group(1).split(',')])
    val_loss_match = re.search(r'val_loss: \[(.*)\]', line)
    if val_loss_match:
        val_loss.extend([float(x) for x in val_loss_match.group(1).split(',')])
    val_accuracy_match = re.search(r'val_accuracy: \[(.*)\]', line)
    if val_accuracy_match:
        val_accuracy.extend([float(x) for x in val_accuracy_match.group(1).split(',')])

# 输出结果
print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)


## 百度Evaluation

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as scio
import tensorflow as tf

# Load the model
model = model # Load your TensorFlow model here

# Read the test data
test_image = pd.read_csv('/home/u200810216/jupyter/kaggle_m3cv/Testing_Info.csv')
test_image_path_list = test_image['Row'].values

eeg_list = list()

labeled_img_list = []
for img in test_image_path_list:
    eeg_list.append('/home/u200810216/jupyter/kaggle_m3cv/test/'+img+'.mat')
    labeled_img_list.append(img)

def load_eeg(eeg_path):
    # Load data
    data = scio.loadmat(eeg_path)
    return data['epoch_data']

pre_list = []
for i in tqdm(range(len(eeg_list))):
    data = load_eeg(eeg_path=eeg_list[i])
    dy_x_data = np.array(data).astype('float32')
    dy_x_data = dy_x_data[np.newaxis, :, :]
    eeg = tf.convert_to_tensor(dy_x_data)
    eeg = eeg[..., np.newaxis]

    out = model(eeg)
    out = np.argmax(out.numpy())

    pre_list.append(int(out)+1)

img_test = pd.DataFrame(labeled_img_list)
img_pre = pd.DataFrame(labeled_img_list)

img_test = img_test.rename(columns = {0:"EpochID"})
img_pre['SubjectID'] = pre_list
pre_info = img_pre['SubjectID'].values
test_info = test_image['SubjectID'].values

result_cnn = list()

for i, j in zip(test_info, pre_info):
    if i == 'None':
        result_cnn.append(j)
    elif int(i[4:])==j :
        print(i[4:])
        result_cnn.append(int(1))
    else:
        result_cnn.append(int(0))

img_test['Prediction'] = result_cnn
img_test.to_csv('result_vit_kaggle.csv', index=False)

  0%|          | 34/52342 [00:07<3:20:07,  4.36it/s]


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io as scio
import tensorflow as tf
from tqdm import tqdm

# Load the model
model = model # Load your TensorFlow model here

# Read the test data
test_image = pd.read_csv('/home/u200810216/jupyter/kaggle_m3cv/Testing_Info.csv')
test_image_path_list = test_image['Row'].values

eeg_list = ['/home/u200810216/jupyter/kaggle_m3cv/test/'+img+'.mat' for img in test_image_path_list]

def load_eeg(eeg_path):
    data = scio.loadmat(eeg_path)
    return data['epoch_data']

# Set batch size
batch_size = 512

# Preallocate lists for results
pre_list = []

# Iterate over dataset with steps of batch_size
for i in tqdm(range(0, len(eeg_list), batch_size)):
    batch_eeg_list = eeg_list[i:i+batch_size]
    batch_data = np.array([load_eeg(eeg_path) for eeg_path in batch_eeg_list]).astype('float32')
    batch_data = batch_data[..., np.newaxis]

    # Convert to tensor and feed to the model
    batch_data_tensor = tf.convert_to_tensor(batch_data)
    batch_out = model(batch_data_tensor)
    batch_pre_list = np.argmax(batch_out.numpy(), axis=-1) + 1

    pre_list.extend(batch_pre_list)

# Post-processing and saving results
img_test = pd.DataFrame(test_image_path_list, columns=["EpochID"])
img_pre = pd.DataFrame(test_image_path_list, columns=["EpochID"])
img_pre['SubjectID'] = pre_list
pre_info = img_pre['SubjectID'].values
test_info = test_image['SubjectID'].values

result_cnn = list()

for i, j in zip(test_info, pre_info):
    if i == 'None':
        result_cnn.append(j)
    elif int(i[4:])==j :
        print(i[4:])
        result_cnn.append(int(1))
    else:
        result_cnn.append(int(0))

img_test['Prediction'] = result_cnn
img_test.to_csv('result_vit_kaggle_roll64.csv', index=False)


100%|██████████| 103/103 [09:19<00:00,  5.43s/it]


29
59
51
76
94
88
29
29
88
26
65
31
88
24
59
55
52
72
91
59
88
94
71
27
52
29
71
39
39
23
29
31
95
29
79
29
94
76
59
44
72
45
59
76
65
29
72
94
35
88
57
72
84
88
44
29
76
29
71
77
84
42
52
91
88
53
59
31
94
76
94
23
23
76
31
88
29
94
25
43
23
36
29
28
29
77
29
42
71
29
88
29
32
25
29
21
32
71
74
32
23
59
42
88
52
29
31
91
29
59
43
58
77
41
21
57
24
91
72
65
29
29
32
31
29
32
38
23
46
35
52
65
52
42
29
78
30
45
29
57
59
24
83
29
71
38
91
83
27
59
94
42
29
52
75
59
29
94
38
24
95
84
83
83
93
29
88
57
54
46
23
77
29
32
57
52
59
88
30
25
83
29
45
71
29
88
24
59
31
24
29
59
29
88
93
94
31
52
59
46
29
40
29
29
27
77
76
23
77
23
23
35
44
76
27
24
57
70
29
60
94
53
59
42
88
94
30
59
29
46
78
29
39
60
84
53
76
32
52
29
52
94
25
27
42
87
72
57
60
71
88
42
29
23
40
43
25
29
30
76
40
58
43
71
77
66
47
32
29
59
78
45
32
57
76
31
77
29
76
42
42
75
52
23
52
71
71
29
23
76
52
56
25
26
91
76
71
83
32
76
94
77
88
70
30
59
77
29
88
32
23
94
52
94
44
28
27
94
94
44
62
83
40
88
29
77
30
29
76
52
29
52
75
5

# Environment Debugging Code

In [ ]:
import tensorflow as tf
print(tf.__version__)


2023-05-06 00:29:55.574764: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 00:29:59.771098: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-06 00:30:22.878649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-06 00:30:22.879038: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

2.11.0


In [ ]:
! pip install tqdm
! sudo apt-get update
import os
from tqdm import tqdm

def install_package_with_apt(package_name):
    os.system(f"sudo apt-get install --allow-change-held-packages --yes {package_name}")

def install_package_with_pip(package_name):
    os.system(f"pip install {package_name}")

apt_packages = [
    "texlive-latex-recommended",
    "dvipng",
    "texlive-latex-extra",
    "texlive-fonts-recommended",
    "cm-super",
]

pip_packages = [
    "Pillow",
    "umap",
    "xgboost",
    "seaborn",
    "SciencePlots",
    "pytorch-tabnet",
    "pycountry",
    "us",
    "uszipcode",
    "torch transformers",
    "geopandas matplotlib pandas",
    "geopandas descartes mapclassify",
    "SciencePlots",
    "geopandas matplotlib pandas",
    "ta",
    "pytrends",
    "chinese-calendar",
    "ephem",
    "category_encoders",
    "fbprophet",
    "statsmodels",
]

for package in tqdm(apt_packages):
    install_package_with_apt(package)

for package in tqdm(pip_packages):
    install_package_with_pip(package)

Defaulting to user installation because normal site-packages is not writeable
Hit:1 https://mirrors.tuna.tsinghua.edu.cn/ubuntu focal InRelease
Get:2 https://mirrors.tuna.tsinghua.edu.cn/ubuntu focal-updates InRelease [114 kB]
Get:3 https://mirrors.tuna.tsinghua.edu.cn/ubuntu focal-backports InRelease [108 kB]
Get:4 https://mirrors.tuna.tsinghua.edu.cn/ubuntu focal-security InRelease [114 kB]
Fetched 336 kB in 1s (580 kB/s)                              
Reading package lists... Done



  0%|          | 0/5 [00:00<?, ?it/s]

Reading package lists...
Building dependency tree...
Reading state information...



 20%|██        | 1/5 [00:01<00:04,  1.21s/it]

texlive-latex-recommended is already the newest version (2019.20200218-1).
The following packages were automatically installed and are no longer required:
  dh-python gir1.2-harfbuzz-0.0 ibverbs-providers icu-devtools libblkid-dev
  libboost-atomic-dev libboost-atomic1.65-dev libboost-atomic1.65.1
  libboost-chrono-dev libboost-chrono1.65-dev libboost-chrono1.65.1
  libboost-container-dev libboost-container1.65-dev libboost-container1.65.1
  libboost-context-dev libboost-context1.65-dev libboost-context1.65.1
  libboost-coroutine-dev libboost-coroutine1.65-dev libboost-coroutine1.65.1
  libboost-date-time-dev libboost-date-time1.65-dev libboost-date-time1.65.1
  libboost-dev libboost-exception-dev libboost-exception1.65-dev
  libboost-fiber-dev libboost-fiber1.65-dev libboost-fiber1.65.1
  libboost-filesystem-dev libboost-filesystem1.65-dev
  libboost-filesystem1.65.1 libboost-graph-dev libboost-graph-parallel-dev
  libboost-graph-parallel1.65-dev libboost-graph-parallel1.65.1
  libboo

...

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 94.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 272.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 335.0 kB/s eta 0:00:00

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [45 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/fbprophet
      creating build/lib/fbprophet/stan_model
      Traceback (most recent call last):
        File "<string>", line 36, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-vogm942y/fbprophet_cacf5a34fc3f4cceb3d36d6a373cef7a/setup.py", line 149, in <module>
          long_description_content_type='text/markdown',
        File "/opt/conda/lib/python3.7/site-packages/setuptools/__init__.py", line 153, in setup
          return distutils.core.setup(**attrs)
        File "/opt/conda/lib/python3.7/distutils/core.py", line 148, in setup
          dist.run_commands()
        File "/opt/conda/lib/python3.7/distutils/dist.py", line 966, in run_commands
   

ERROR: Could not build wheels for fbprophet, which is required to install pyproject.toml-based projects

 95%|█████████▌| 20/21 [01:57<00:18, 18.43s/it]

  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732018 sha256=d65d09aed1a8c0895788557e771f919ff4ce122a0e9d61bbe01a490d31ce9456
  Stored in directory: /home/u200810216/.cache/pip/wheels/1e/5a/c0/fca16a6d08089e38ac5a71f3f8ea7a9a12b3356ff3debd8c59
Successfully built pymeeus
Failed to build fbprophet
Defaulting to user installation because normal site-packages is not writeable


100%|██████████| 21/21 [02:00<00:00,  5.74s/it]


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow-gpu
# !pip uninstall SciencePlots -y
# !pip install SciencePlots

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import matplotlib.pyplot as plt
print(plt.style.available)

In [ ]:
import sys

print("当前Python解释器的地址：", sys.executable)


当前Python解释器的地址： /opt/conda/bin/python


In [ ]:
!jupyter notebook list


Currently running servers:
http://0.0.0.0:8888/proxy/service/2d3d1f0804005/ :: /home/u200810216/jupyter


In [ ]:
# 确保已安装jupyter_contrib_nbextensions。如果您还没有安装它，可以使用以下命令进行安装：
# 建议打开putty su - u200810216切换到普通用户，然后执行以下命令
# pip install jupyter_contrib_nbextensions
# jupyter contrib nbextension install --user
# jupyter nbextension enable toc2/main
# jupyter nbextension enable toc2/main --user --section=notebook

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install SciencePlots

# Model Parameter Search

In [ ]:
os.chdir('/home/u200810216/jupyter/')

## 简陋版本

In [ ]:
import os
import itertools
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Reshape
from tensorflow.keras.layers import AveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pickle
import json

# Set up your model
def create_model(use_layer_7, use_layer_8):
    model = Sequential()

    #layer1
    model.add(Conv2D(128, kernel_size=(n_ch, 1), padding="valid", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(1, 2)))
    # model.add(Dropout(0.1))

    ## Layer 2
    model.add(Conv2D(256, kernel_size=(128, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(1, 2)))
    # model.add(Dropout(0.1))

    ## Layer 3
    model.add(Conv2D(512, kernel_size=(256, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(1, 2)))
    # model.add(Dropout(0.1))

    ## Layer 4
    model.add(Conv2D(1024, kernel_size=(512, 2), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(4, 1)))
    model.add(Flatten())


    # layer 5
    model.add(Dense(192))
    model.add(Activation('relu'))
    # model.add(Dropout(0.4))


    model.add(Reshape((model.output_shape[1], 1)))
    # layer 6
    model.add(LSTM(192))

    if use_layer_7:
        model.add(Reshape((model.output_shape[1], 1)))
        model.add(LSTM(192))

    if use_layer_8:
        model.add(Dense(192))

    # Add output layer
    # ...

    # layer 9
    model.add(Dense(192))


    #output layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.summary()
    return model

# Set up experiment parameters
batch_sizes = [128, 256]
learning_rates = [0.01, 0.1, 1]
decays = [0.01, 0.1, 1]
shuffles = [True, False]
use_layer_7_options = [True, False]
use_layer_8_options = [True, False]
epochs = 200

# Calculate total combinations
total_combinations = len(batch_sizes) * len(learning_rates) * len(decays) * len(shuffles) * len(use_layer_7_options) * len(use_layer_8_options)
print(f'Total parameter combinations: {total_combinations}')

# Create combinations of parameters
combinations = list(itertools.product(batch_sizes, learning_rates, decays, shuffles, use_layer_7_options, use_layer_8_options))

# Set up experiment state
experiment_state_path = 'experiment_state.pickle'
if os.path.exists(experiment_state_path):
    with open(experiment_state_path, 'rb') as f:
        experiment_state = pickle.load(f)
    start_index = experiment_state['completed_combinations']
else:
    experiment_state = {'completed_combinations': 0}
    start_index = 0
try:
    # Iterate through combinations
    for idx, (batch_size, lr, decay, shuffle, use_layer_7, use_layer_8) in enumerate(combinations[start_index:], start_index):
        # Set up model name and file names
        model_name = f'model_LSTMCNN_epochs200_batch{batch_size}_shuffle{shuffle}_lr{lr}_decay{decay}_layer7_{use_layer_7}_layer8_{use_layer_8}'
        log_dir = f"logs/fit/{model_name}"
        os.makedirs(log_dir, exist_ok=True)

        checkpoint_path = os.path.join(log_dir, f'{model_name}_checkpoint.h5')
        final_model_path = os.path.join(log_dir, f'{model_name}_final.h5')
        acc_plot_path = os.path.join(log_dir, f'{model_name}_accuracy.png')
        loss_plot_path = os.path.join(log_dir, f'{model_name}_loss.png')
        history_data_path = os.path.join(log_dir, f'{model_name}_history.json')

        # Create model
        model = create_model(use_layer_7, use_layer_8)

        # Compile the model
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr, decay=decay),
                      metrics=['accuracy', 'FalsePositives', 'FalseNegatives'])

        # Set up callbacks
        log_dir = f"logs/fit/{model_name}"
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
        checkpoint_callback = ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=False,
            save_best_only=True,
            monitor='val_loss',
            verbose=1,
        )


        # Check if the checkpoint file exists
        if os.path.exists(checkpoint_path):
            # Load the model from the checkpoint file
            model = tf.keras.models.load_model(checkpoint_path)
            print(f"Loaded model from {checkpoint_path}")
            initial_epoch = model.optimizer.iterations.numpy() // (len(x_train) // batch_size)
        else:
            print(f"No checkpoint file found at {checkpoint_path}, training from scratch.")
            initial_epoch = 0


        # Train the model
        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            shuffle=shuffle,
                            verbose=1,
                            validation_split=0.2,
                            callbacks=[tensorboard_callback, checkpoint_callback],
                            initial_epoch=initial_epoch)

        # Save the final model
        model.save(final_model_path)

        # Evaluate the model
        score_train = model.evaluate(x_train, y_train, verbose=0)
        score_test = model.evaluate(x_test, y_test, verbose=0)
        print("mean train accuracy is:   ", (score_train[1]))
        print("mean test accuracy is:   ", (score_test[1]))
        print("FAR is:   ", (score_train[2] + score_test[2]) / (np.size(y_train, 0) + np.size(y_test, 0)))
        print("FRR is:   ", (score_train[3] + score_test[3]) / (np.size(y_train, 0) + np.size(y_test, 0)))

        # Plot and save accuracy
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.grid(True)
        plt.savefig(acc_plot_path, dpi=600)
        plt.show()

        # Plot and save loss
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.grid(True)
        plt.savefig(loss_plot_path, dpi=600)
        plt.show()
        # Save history data to a JSON file

        with open(history_data_path, 'w') as history_file:
            json.dump(history.history, history_file)
        # Update experiment state
        experiment_state['completed_combinations'] = idx + 1
        with open(experiment_state_path, 'wb') as f:
            pickle.dump(experiment_state, f)




except KeyboardInterrupt:

    print("\n\n\nInterrupted, saving experiment state...")
    with open(experiment_state_path, 'wb') as f:
        pickle.dump(experiment_state, f)
    print("Experiment state saved. You can resume the experiment later.")

## TensorBoard调参版本

In [ ]:
import os
import itertools
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, Reshape
from tensorflow.keras.layers import AveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pickle
import json
from tensorboard.plugins.hparams import api as hp

HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([128, 256]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.01, 0.1, 1.0]))
HP_DECAY = hp.HParam('decay', hp.Discrete([0.01, 0.1, 1.0]))
HP_SHUFFLE = hp.HParam('shuffle', hp.Discrete([True, False]))
HP_USE_LAYER_7 = hp.HParam('use_layer_7', hp.Discrete([True, False]))
HP_USE_LAYER_8 = hp.HParam('use_layer_8', hp.Discrete([True, False]))
batch_sizes = HP_BATCH_SIZE.domain.values
learning_rates = HP_LEARNING_RATE.domain.values
decays = HP_DECAY.domain.values
shuffles = HP_SHUFFLE.domain.values
use_layer_7_options = HP_USE_LAYER_7.domain.values
use_layer_8_options = HP_USE_LAYER_8.domain.values

epochs = 10
params = [HP_BATCH_SIZE, HP_LEARNING_RATE, HP_DECAY, HP_SHUFFLE, HP_USE_LAYER_7, HP_USE_LAYER_8]

def create_model(hparams):
    use_layer_7 = hparams[HP_USE_LAYER_7]
    use_layer_8 = hparams[HP_USE_LAYER_8]

    model = Sequential()

    #layer1
    model.add(Conv2D(128, kernel_size=(n_ch, 1), padding="valid", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(1, 2)))
    # model.add(Dropout(0.1))

    ## Layer 2
    model.add(Conv2D(256, kernel_size=(128, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(1, 2)))
    # model.add(Dropout(0.1))

    ## Layer 3
    model.add(Conv2D(512, kernel_size=(256, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(1, 2)))
    # model.add(Dropout(0.1))

    ## Layer 4
    model.add(Conv2D(1024, kernel_size=(512, 2), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((model.output_shape[3],model.output_shape[2],model.output_shape[1])))
    # model.add(AveragePooling2D(pool_size=(4, 1)))
    model.add(Flatten())


    # layer 5
    model.add(Dense(192))
    model.add(Activation('relu'))
    # model.add(Dropout(0.4))


    model.add(Reshape((model.output_shape[1], 1)))
    # layer 6
    model.add(LSTM(192))

    if use_layer_7:
        model.add(Reshape((model.output_shape[1], 1)))
        model.add(LSTM(192))

    if use_layer_8:
        model.add(Dense(192))

    # Add output layer
    # ...

    # layer 9
    model.add(Dense(192))


    #output layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.summary()
    return model
    return model

def run_experiment(hparams, logdir):
    model_name = f'model_LSTMCNN_epochs200_batch{hparams[HP_BATCH_SIZE]}_shuffle{hparams[HP_SHUFFLE]}_lr{hparams[HP_LEARNING_RATE]}_decay{hparams[HP_DECAY]}_layer7_{hparams[HP_USE_LAYER_7]}_layer8_{hparams[HP_USE_LAYER_8]}'
    os.makedirs(logdir, exist_ok=True)

    checkpoint_path = os.path.join(logdir, f'{model_name}_checkpoint.h5')
    final_model_path = os.path.join(logdir, f'{model_name}_final.h5')
    acc_plot_path = os.path.join(logdir, f'{model_name}_accuracy.png')
    loss_plot_path = os.path.join(logdir, f'{model_name}_loss.png')
    history_data_path = os.path.join(logdir, f'{model_name}_history.json')

    # Create model
    model = create_model(hparams)

    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=hparams[HP_LEARNING_RATE], decay=hparams[HP_DECAY]),
                  metrics=['accuracy', 'FalsePositives', 'FalseNegatives'])

    # Set up callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
    hparams_callback = hp.KerasCallback(logdir, hparams)
    checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=False,
        save_best_only=True,
        monitor='val_loss',
        verbose=1,
    )

    # Check if the checkpoint file exists
    if os.path.exists(checkpoint_path):
        # Load the model from the checkpoint file
        model = tf.keras.models.load_model(checkpoint_path)
        print(f"Loaded model from {checkpoint_path}")
        initial_epoch = model.optimizer.iterations.numpy() // (len(x_train) // hparams[HP_BATCH_SIZE])
    else:
        print(f"No checkpoint file found at {checkpoint_path}, training from scratch.")
        initial_epoch = 0

    # Train the model
    history = model.fit(x_train, y_train,
                        batch_size=hparams[HP_BATCH_SIZE],
                        epochs=epochs,
                        shuffle=hparams[HP_SHUFFLE],
                        verbose=1,
                        validation_split=0.2,
                        callbacks=[tensorboard_callback, hparams_callback, checkpoint_callback],
                        initial_epoch=initial_epoch)

    # Save the final model
    model.save(final_model_path)

    # Evaluate the model
    score_train = model.evaluate(x_train, y_train, verbose=0)
    score_test = model.evaluate(x_test, y_test, verbose=0)
    print("mean train accuracy is:   ", (score_train[1]))
    print("mean test accuracy is:   ", (score_test[1]))
    print("FAR is:   ", (score_train[2] + score_test[2]) / (np.size(y_train, 0) + np.size(y_test, 0)))
    print("FRR is:   ", (score_train[3] + score_test[3]) / (np.size(y_train, 0) + np.size(y_test, 0)))

    # Plot and save accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.grid(True)
    plt.savefig(acc_plot_path, dpi=600)
    plt.show()
    # Plot and save loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.grid(True)
    plt.savefig(loss_plot_path, dpi=600)
    plt.show()

    # Save history data to a JSON file
    with open(history_data_path, 'w') as history_file:
        json.dump(history.history, history_file)

    return history


# Generate the combinations list
combinations = list(itertools.product(batch_sizes, learning_rates, decays, shuffles, use_layer_7_options, use_layer_8_options))

try:
    start_index = 0
    experiment_state = {'completed_combinations': 0}
    experiment_state_path = 'experiment_state.pkl'

    # Check if the experiment state file exists
    if os.path.exists(experiment_state_path):
        with open(experiment_state_path, 'rb') as f:
            experiment_state = pickle.load(f)
        start_index = experiment_state['completed_combinations']

    # Iterate through combinations
    for idx, hparams_tuple in enumerate(combinations[start_index:], start_index):
        hparams = {param: value for param, value in zip(params, hparams_tuple)}
        logdir = f"logs/hparam_tuning/model_LSTMCNN_epochs200_batch{hparams[HP_BATCH_SIZE]}_shuffle{hparams[HP_SHUFFLE]}_lr{hparams[HP_LEARNING_RATE]}_decay{hparams[HP_DECAY]}_layer7_{hparams[HP_USE_LAYER_7]}_layer8_{hparams[HP_USE_LAYER_8]}"
        run_experiment(hparams, logdir)

        # Update experiment state
        experiment_state['completed_combinations'] = idx + 1
        with open(experiment_state_path, 'wb') as f:
            pickle.dump(experiment_state, f)

            # write bubble sort function






except KeyboardInterrupt:
    print("\n\n\nInterrupted, saving experiment state...")
    with open(experiment_state_path, 'wb') as f:
        pickle.dump(experiment_state, f)
    print("Experiment state saved. You can resume the experiment later.")
